<a href="https://colab.research.google.com/github/hand-e-fr/OpenHosta/blob/doc/docs/openhosta_phi4_mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenHosta basic example with a Local Phi-4 or gpt-4o

This colab demonstrate simple use cases of OpenHosta. If you do not have an OpenAI (or other) API KEY, you can run the first part **Install a local Phi-4**. Otherwise, directly jump to step 2: **Basic Usage**.

## Install a local Phi-4

In [1]:
!apt install -y screen

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  byobu | screenie | iselect ncurses-term
The following NEW packages will be installed:
  screen
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 672 kB of archives.
After this operation, 1,029 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 screen amd64 4.9.0-1 [672 kB]
Fetched 672 kB in 1s (1,062 kB/s)
Selecting previously unselected package screen.
(Reading database ... 124947 files and directories currently installed.)
Preparing to unpack .../screen_4.9.0-1_amd64.deb ...
Unpacking screen (4.9.0-1) ...
Setting up screen (4.9.0-1) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
# Install vLLM from pip:
!pip install vllm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.9/396.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.

In [ ]:
#!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
# !screen -dmS ollama ollama serve
# Load and run the model:
!screen -dmS vllm serve "microsoft/Phi-4-mini-instruct"

In [ ]:
# Can take 5 minutes
!ollama run phi4 hello --verbose  2>&1 | grep -E "([^0]0%|Bonjour|:)"

pulling dde5aa3fc5ff...   0% ▕                ▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕                ▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕                ▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕                ▏ 334 KB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕                ▏ 4.9 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...  10% ▕█               ▏ 194 MB/2.0 GB  167 MB/s     10spulling manifest 
pulling dde5aa3fc5ff...  10% ▕█               ▏ 200 MB/2.0 GB  167 MB/s     10spulling manifest 
pulling dde5aa3fc5ff...  20% ▕███             ▏ 409 MB/2.0 GB  167 MB/s      9spulling manifest 
pulling dde5aa3fc5ff...  30% ▕████            ▏ 599 MB/2.0 GB  241 MB/s      5spulling manifest 
pulling dde5aa3fc5ff...  30% ▕████            ▏ 608 MB/2.0 GB  241 MB/s      5spulling manifest 
pulling dde5aa3fc5ff...  40% ▕

In [ ]:
!pip install OpenHosta

## Basic Usage of OpenHosta

### Configure the LLM that you want to use

In [ ]:
from OpenHosta import config

# If you have an OpenAI API key you can use it like this:
# Use default model (gpt-4o) through API key
# config.set_default_apiKey(<<YOUR API KEY>>)

# Comment this if you use OpenAI models
# Use Microsoft local Phi-4 through ollama
my_model=config.Model(
    base_url="http://localhost:11434/v1/chat/completions",
    model="phi4", api_key="none", timeout=120
)
config.set_default_model(my_model)


### Emulate functions using the seleted LLM

In [ ]:
from OpenHosta import emulate

In [ ]:
def translate(text:str, language:str)->str:
    """
    This function translates the text in the “text” parameter into the language specified in the “language” parameter.
    """
    return emulate()

result = translate("Hello World!", "French")
print(result)

Bonjour le monde!


In [ ]:
# You can select another model like this
#my_model = config.Model(
#    model="gpt-4o-mini",
#    base_url="https://api.openai.com/v1/chat/completions",
#    api_key=<<API KEY>>
#)

In [ ]:
def find_name_age(sentence:str, id:dict)->dict:
    """
    This function find in a text the name and the age of a personn.

    Args:
        sentence: The text in which to search for information
        id: The dictionary to fill in with information

    Return:
        A dictionary identical to the one passed in parameter, but filled with the information.
        If the information is not found, fill with the values with “None”.
    """
    return emulate(model=my_model)

return_dict = {"name": "", "age": ""}
result = find_name_age("Hello, I'm John Wick, i'm 30 and I live in New York", return_dict)
print(result)

{'name': 'John Wick', 'age': '30'}


### Specify advanced return types

In [ ]:
from typing import Dict, Tuple, List

def analyze_text(text: str) -> Dict[str, List[Tuple[int, str]]]:
    """Analyze text to map each word to a list of tuples containing word length and word."""
    return emulate()

# Example usage
analysis = analyze_text("Hello, World!")

print(analysis)
print(type(analysis))

{'hello': [[5, 'Hello']], 'world': [[5, 'World']]}
<class 'dict'>


### Specify pydantic return strucures

OpenHosta is compatible with pydantic. You can specify pydantic input and output types and OpenHosta will propagate schema and Field documentation to the LLM.

In [ ]:
!pip install pydantic

In [ ]:
from pydantic import BaseModel, Field

class Personn(BaseModel):
    name: str = Field(..., description = "The full name")
    age: int

def find_name_age_pydantic(sentence:str)->Personn:
    """
    This function find in a text the name and the age of a personn.

    Args:
        sentence: The text in which to search for information

    Return:
        A Pydantic model, but filled with the information.
        If the information is not found, fill with the values with “None”.
    """
    return emulate()

result = find_name_age_pydantic("Luke Skywalker is very surprising: he's only 27 when he becomes a Jedi.")
print(result)

name='Luke Skywalker' age=27


### Limitations

The emulation is limited by the LLM capabilities. Try to have it count r in strawberrry and you will get into troubles ;-).
Make sure the LLM is capable and not alucinating before you implement an emulated function.

In [ ]:
def find_occurence_of_a_word(word :str, text: str) -> int:
    """
    This function takes a word and a text and returns
    the number of times the word appears in the text.
    """
    return emulate()

find_occurence_of_a_word("Hello", "Hello World Hello!")


2